In [9]:
import numpy as np
import pandas as pd
import requests
import math
import json
import statistics
import pprint
import scipy
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from Secrets.binance_secrets import BINANCE_API_KEY
from Secrets.binance_secrets import BINANCE_SECRET
from binance.client import Client

In [10]:
client = Client(BINANCE_API_KEY, BINANCE_SECRET)

In [14]:
client.ping()

{}

In [29]:
candle = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1SECOND)
print(candle)

[[1736578215000, '94324.24000000', '94324.24000000', '94324.24000000', '94324.24000000', '0.00840000', 1736578215999, '792.32361600', 3, '0.00000000', '0.00000000', '0'], [1736578216000, '94324.24000000', '94324.24000000', '94324.24000000', '94324.24000000', '0.03455000', 1736578216999, '3258.90249200', 2, '0.00000000', '0.00000000', '0'], [1736578217000, '94324.24000000', '94324.24000000', '94324.24000000', '94324.24000000', '0.00000000', 1736578217999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1736578218000, '94324.25000000', '94324.25000000', '94324.25000000', '94324.25000000', '0.00011000', 1736578218999, '10.37566750', 1, '0.00011000', '10.37566750', '0'], [1736578219000, '94324.24000000', '94324.24000000', '94323.54000000', '94323.54000000', '0.01781000', 1736578219999, '1679.91380500', 49, '0.00000000', '0.00000000', '0'], [1736578220000, '94323.53000000', '94323.53000000', '94322.62000000', '94322.63000000', '0.00439000', 1736578220999, '414.07809530', 28, '0.00053000

In [25]:
candles = client.get_historical_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_30MINUTE, limit=1, start_str="1 Dec, 2024", end_str="1 Jan, 2025")
print(candles)

[[1733011200000, '96407.99000000', '96608.00000000', '96350.13000000', '96601.41000000', '189.73428000', 1733012999999, '18314709.13016570', 58518, '94.61803000', '9132767.34515900', '0']]
